# In The Name Of GOD
# 🖼️ Visual Search Engine with CLIP & FAISS
Ever wanted to find an image using another image? This project lets you do just that. Upload a picture, and the AI-powered engine will instantly find the most visually similar images from a library of thousands.

The entire project runs in a single Google Colab notebook.

## 1. Setup and Installations

In [1]:
!pip install -q transformers datasets faiss-cpu gradio

## 2. Load Model and Dataset

We load our model and a dataset of 8,000 images.

In [1]:
!rm -rf ~/.cache/huggingface/datasets

In [2]:
!pip install -U datasets==2.15.0

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)

train_dataset = load_dataset("frgfm/imagenette", "320px", split="train")
image_subset = train_dataset.select(range(8000))

validation_dataset = load_dataset("frgfm/imagenette", "320px", split="validation")

class_labels = image_subset.features['label'].names

print(f"Search index created with {len(image_subset)} images from the train set.")
print(f"Example images will be loaded from the {len(validation_dataset)} images in the validation set.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Generating train split:   0%|          | 0/9469 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3925 [00:00<?, ? examples/s]

Search index created with 8000 images from the train set.
Example images will be loaded from the 3925 images in the validation set.


## 3. Feature Extraction

In [4]:
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

def extract_features(images):
    inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)

    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    return image_features.cpu().numpy()

all_embeddings = []
for i in tqdm(range(0, len(image_subset), 32)):
    batch = image_subset[i:i+32]['image']
    embeddings = extract_features(batch)
    all_embeddings.append(embeddings)

all_embeddings = np.concatenate(all_embeddings).astype('float32')

print(f"Shape of our feature matrix: {all_embeddings.shape}")

  0%|          | 0/250 [00:00<?, ?it/s]

Shape of our feature matrix: (8000, 512)


## 4. Build the FAISS Index

In [5]:
import faiss

embedding_dim = all_embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(all_embeddings)

print(f"FAISS index created with {index.ntotal} vectors.")

FAISS index created with 8000 vectors.


## 5. Create the Search Function and Final UI

In [ ]:
import gradio as gr

def search_similar_images(query_image, k=9):
    query_embedding = extract_features([query_image])
    distances, indices = index.search(query_embedding, k)
    results = []
    for i in indices[0]:
        image = image_subset[int(i)]['image']
        label_index = image_subset[int(i)]['label']
        label_name = class_labels[label_index]
        results.append((image, label_name))
    return results

example_images = [
    [validation_dataset[10]['image']],
    [validation_dataset[500]['image']],
    [validation_dataset[1000]['image']],
    [validation_dataset[1500]['image']]
]

iface = gr.Interface(
    fn=search_similar_images,
    inputs=gr.Image(type="pil", label="Upload an Image to Search"),
    outputs=gr.Gallery(label="Search Results", rows=3, columns=3, object_fit="contain"),
    title="✨ Visual Search Engine ✨",
    description="Discover visually similar images. Upload your own image or click an example from the test set below.",
    examples=example_images,
    theme='gradio/soft'  
)

iface.launch(debug=True, share=True)

theme_schema%400.0.3.json: 0.00B [00:00, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6a7eba9b912f9e87a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Developed By Eiliya Mohebi For Educational Purposes (Filoger Advanced CV Course Final Project).